In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import swifter
import time
import warnings

from src import meteorological_api
import src.utils as utils
import multiprocessing as mp
import os

In [8]:
warnings.filterwarnings('ignore')

In [9]:
response_var = pd.read_parquet('../data/interim/storm_outages_2014_2023b.parquet')
counties = gpd.read_parquet('../data/external/county.parquet')

In [10]:
counties['fips_code_id'] = counties['STATEFP'] + counties['COUNTYFP'] 
outages_storms = response_var[response_var.storm_caused_outage==1]

In [11]:
# Leo response_var y os.meteorological
response_var = pd.read_parquet('../data/interim/storm_outages_2014_2023b.parquet')
outages_storms = response_var[response_var.storm_caused_outage==1]

# Los hago comparables
meteorological_ids_data = set(os.listdir('../data/raw/meteorological')) # Estos ya los descargue
unique_efi = set(outages_storms.episode_fips_id + '.json') # Estos son los que existen y puedo descargar

In [12]:
#unique_efi - meteorological_ids_data # La resta me dice cuales no he descargado de los que existen y puedo descargar. 

In [16]:
element_nb = 200
random_seed = 42
meaning_dict = {'begin_datetime': 'storm_start', 'end_datetime': 'storm_end', 'run_start_time_min': 'outage_start', 'run_start_time_max': 'outage_end'}


actual_len = len(unique_efi - meteorological_ids_data)
len_list = 0
while actual_len > 0 and len_list < 5:
    outages_storms_candidates = outages_storms[(outages_storms.episode_fips_id + '.json').isin(list(unique_efi-meteorological_ids_data))]
    #sample_outages_storms = outages_storms_candidates.sample(element_nb, random_state=random_seed)
    sample_outages_storms = outages_storms_candidates.sample(element_nb)
    sample_outages_storms = sample_outages_storms.rename(columns=meaning_dict)
    sample_outages_storms['one_day_before_storm'] = (sample_outages_storms['storm_start'] - pd.Timedelta(1, unit='D')).dt.date
    sample_outages_storms['one_day_after_storm'] = (sample_outages_storms['storm_end'] + pd.Timedelta(1, unit='D')).dt.date
    sample_outages_storms = sample_outages_storms.merge(counties[['fips_code_id', 'geometry']], on='fips_code_id')
    sample_outages_storms = gpd.GeoDataFrame(sample_outages_storms)
    sample_outages_storms['cntroid'] = sample_outages_storms.geometry.centroid
    sample_outages_storms['url__cntroid'] = sample_outages_storms.apply(meteorological_api.get_url_params_from_row_point, desired_point='cntroid', axis=1)
    sample_outages_storms = sample_outages_storms[sample_outages_storms['url__cntroid'].notna()]
    t0 = time.perf_counter()
    cpu_counts = mp.cpu_count()
    print(f'Available CPUs: {cpu_counts}')
    # Request meteorological data
    with mp.Pool(cpu_counts) as pool:
        sample_outages_storms['meteorological_info__cntroid'] = pool.map(
            meteorological_api.get_meteorological_info, sample_outages_storms.url__cntroid
        )
    
    t1 = time.perf_counter()
    print(f'time it took {t1-t0}')
    # Save data
    for row in sample_outages_storms.iterrows():
        meteorological_api.save_meteorological_info_from_row_point(row)
    print('Done with saving.')
    # Wait.
    time.sleep(5)
    meteorological_ids_data = set(os.listdir('../data/raw/meteorological')) # Estos ya los descargue
    unique_efi = set(outages_storms.episode_fips_id + '.json') # Estos son los que existen y puedo descargar
    current_len = actual_len
    actual_len = len(unique_efi - meteorological_ids_data)
    print(current_len, actual_len)
    if current_len == actual_len:
        len_list += 1
    else:
        len_list=0

Available CPUs: 20
time it took 131.42945069499547
Saving info at: /mnt/c/Users/52333/Documents/projects/dynamic-rythms/src/../data/raw/meteorological/171114_01127.json
Saving info at: /mnt/c/Users/52333/Documents/projects/dynamic-rythms/src/../data/raw/meteorological/130410_45085.json
Saving info at: /mnt/c/Users/52333/Documents/projects/dynamic-rythms/src/../data/raw/meteorological/141564_06065.json
Saving info at: /mnt/c/Users/52333/Documents/projects/dynamic-rythms/src/../data/raw/meteorological/181799_12086.json
Saving info at: /mnt/c/Users/52333/Documents/projects/dynamic-rythms/src/../data/raw/meteorological/170380_48479.json
Saving info at: /mnt/c/Users/52333/Documents/projects/dynamic-rythms/src/../data/raw/meteorological/139466_05091.json
Saving info at: /mnt/c/Users/52333/Documents/projects/dynamic-rythms/src/../data/raw/meteorological/156839_54043.json
Saving info at: /mnt/c/Users/52333/Documents/projects/dynamic-rythms/src/../data/raw/meteorological/181204_12083.json
Savin

Process ForkPoolWorker-624:
Process ForkPoolWorker-637:
Process ForkPoolWorker-622:
Process ForkPoolWorker-633:
Process ForkPoolWorker-640:
Process ForkPoolWorker-636:
Process ForkPoolWorker-631:
Process ForkPoolWorker-623:
Process ForkPoolWorker-627:
Process ForkPoolWorker-634:
Process ForkPoolWorker-629:
Process ForkPoolWorker-625:
Process ForkPoolWorker-635:
Process ForkPoolWorker-639:
Process ForkPoolWorker-638:
Process ForkPoolWorker-632:


KeyboardInterrupt: 

In [13]:
element_nb = 200
random_seed = 42
outages_storms_candidates = outages_storms[(outages_storms.episode_fips_id + '.json').isin(list(unique_efi))]
sample_outages_storms = outages_storms_candidates.sample(element_nb, random_state=random_seed)
sample_outages_storms = sample_outages_storms.rename(
    columns=meaning_dict
)
if True:
    sample_outages_storms['one_day_before_storm'] = (sample_outages_storms['storm_start'] - pd.Timedelta(1, unit='D')).dt.date
    sample_outages_storms['one_day_after_storm'] = (sample_outages_storms['storm_end'] + pd.Timedelta(1, unit='D')).dt.date
    sample_outages_storms = sample_outages_storms.merge(counties[['fips_code_id', 'geometry']], on='fips_code_id')
    sample_outages_storms = gpd.GeoDataFrame(sample_outages_storms)
    sample_outages_storms['cntroid'] = sample_outages_storms.geometry.centroid
    sample_outages_storms['url__cntroid'] = sample_outages_storms.apply(meteorological_api.get_url_params_from_row_point, desired_point='cntroid', axis=1)
    sample_outages_storms = sample_outages_storms[sample_outages_storms['url__cntroid'].notna()]


In [14]:
sample_outages_storms

,EPISODE_ID,fips_code_id,episode_description,storm_start,storm_end,storm_duration,episode_fips_id,storm_caused_outage,outage_index_id,outage_start_minus_storm_start,...,outage_start_minus_storm_end,outage_end_minus_storm_start,outage_duration,outage_start,outage_end,one_day_before_storm,one_day_after_storm,geometry,cntroid,url__cntroid
0,185200,05087,A ridge of high pressure in the middle and upp...,2023-08-19 10:00:00,2023-08-26 19:00:00,177.000000,185200_05087,1.0,05087__0006,1.239583,...,-6.135417,1.281250,0.041667,2023-08-20 15:45:00,2023-08-20 16:45:00,2023-08-18,2023-08-27,"POLYGON ((-93.96063 35.83326, -93.95187 35.833...",POINT (-93.7246 36.01104),https://power.larc.nasa.gov/api/temporal/hourl...
7,177493,06041,An atmospheric river contributed to flooding c...,2023-01-14 09:37:00,2023-01-14 20:12:00,10.583333,177493_06041,1.0,06041__0065,0.276389,...,-0.164583,0.526389,0.250000,2023-01-14 16:15:00,2023-01-14 22:15:00,2023-01-13,2023-01-15,"MULTIPOLYGON (((-122.44476 37.86128, -122.4385...",POINT (-122.72382 38.07353),https://power.larc.nasa.gov/api/temporal/hourl...
11,182649,42111,Two lines of thunderstorms pushed from the Gre...,2023-08-24 23:15:00,2023-08-25 02:03:00,2.800000,182649_42111,1.0,42111__0014,0.375000,...,0.258333,0.416667,0.041667,2023-08-25 08:15:00,2023-08-25 09:15:00,2023-08-23,2023-08-26,"POLYGON ((-79.39876 39.90054, -79.38762 39.915...",POINT (-79.02825 39.97255),https://power.larc.nasa.gov/api/temporal/hourl...
13,165984,36103,Low pressure passes to our north and west and ...,2022-02-18 06:40:00,2022-02-18 07:20:00,0.666667,165984_36103,1.0,36103__0122,0.076389,...,0.048611,0.430556,0.354167,2022-02-18 08:30:00,2022-02-18 17:00:00,2022-02-17,2022-02-19,"MULTIPOLYGON (((-72.03799 41.25351, -72.02944 ...",POINT (-72.84286 40.87015),https://power.larc.nasa.gov/api/temporal/hourl...
15,146319,39059,An advancing warm front promoted heavy rain an...,2020-03-28 14:36:00,2020-03-28 23:55:00,9.316667,146319_39059,1.0,39059__0000,1.256250,...,0.868056,1.266667,0.010417,2020-03-29 20:45:00,2020-03-29 21:00:00,2020-03-27,2020-03-29,"POLYGON ((-81.72571 39.98197, -81.7211 40.0626...",POINT (-81.49427 40.05209),https://power.larc.nasa.gov/api/temporal/hourl...
19,126736,36105,An intense cold front move through central New...,2018-05-15 13:44:00,2018-05-15 14:40:00,0.933333,126736_36105,1.0,36105__0017,0.250694,...,0.211806,3.521528,3.270833,2018-05-15 19:45:00,2018-05-19 02:15:00,2018-05-14,2018-05-16,"POLYGON ((-75.14468 41.85146, -74.99725 41.918...",POINT (-74.76816 41.71648),https://power.larc.nasa.gov/api/temporal/hourl...
22,145804,13215,A strong cold front associated with a deep upp...,2020-01-11 04:26:00,2020-01-11 20:55:00,16.483333,145804_13215,1.0,13215__0022,0.752778,...,0.065972,0.867361,0.114583,2020-01-11 22:30:00,2020-01-12 01:15:00,2020-01-10,2020-01-12,"POLYGON ((-85.07968 32.6081, -85.02619 32.6079...",POINT (-84.87626 32.51027),https://power.larc.nasa.gov/api/temporal/hourl...
25,115200,40123,A line of storms fired along and just ahead of...,2017-05-18 10:53:00,2017-05-19 02:00:00,15.116667,115200_40123,1.0,40123__0003,0.650694,...,0.020833,0.723611,0.072917,2017-05-19 02:30:00,2017-05-19 04:15:00,2017-05-17,2017-05-20,"POLYGON ((-96.93252 34.68042, -96.93244 34.854...",POINT (-96.68399 34.72832),https://power.larc.nasa.gov/api/temporal/hourl...
29,138154,55137,A line of severe thunderstorms moved across mu...,2019-07-19 17:47:00,2019-07-19 21:30:00,3.716667,138154_55137,1.0,55137__0001,0.946528,...,0.791667,1.186111,0.239583,2019-07-20 16:30:00,2019-07-20 22:15:00,2019-07-18,2019-07-20,"POLYGON ((-89.59803 44.11198, -89.5977 44.1560...",POINT (-89.24298 44.11316),https://power.larc.nasa.gov/api/temporal/hourl...
33,177269,37063,Showers and thunderstorms developed along a pr...,2023-01-12 18:55:00,2023-01-12 21:50:00,2.916667,177269_37063,1.0,37063__0088,0.430556,...,0.309028,0.628472,0.197917,2023-01-13 05:15:00,2023-01-13 10:00:00,2023-01-11,2023-01-13,"POLYGON ((-79.00954 35.90221, -79.0

In [10]:
def get_url_params_from_row_point(storm_outage_row, desired_point=None):
    lon = storm_outage_row[desired_point].x
    lat = storm_outage_row[desired_point].y
    start = storm_outage_row.one_day_before_storm
    end = storm_outage_row.one_day_after_storm
    identifier = storm_outage_row['episode_fips_id']
    save_path = f'../data/raw/meteorological/{identifier}.json'
    filexists = utils.check_if_filepath_exists(save_path)
    if not filexists:
        url = meteorological_api.get_api_url(lat=lat, lon=lon, datetime_start=start, datetime_end=end)
        return url
    return np.nan

def save_meteorological_info_from_row_point(storm_outage_row_iterated):
    idx, storm_outage_row = storm_outage_row_iterated
    identifier = storm_outage_row['episode_fips_id']
    meteorological_info = storm_outage_row['meteorological_info__cntroid']
    validate = meteorological_info.get('properties')
    save_path = f'../data/raw/meteorological/{identifier}.json'
    filexists = utils.check_if_filepath_exists(save_path)
    if (not filexists) and (validate is not None):
        meteorological_api.save_meteorological_information(information=meteorological_info, path=save_path)
        print(f'Saving info at: {save_path}')
        return 1
    return np.nan

In [7]:
#element_nb = 2200

In [8]:
condition = 100

In [12]:
import multiprocessing as mp


In [15]:
while condition:
    # Transform data
    print('Starting')
    element_nb += 200
    sample_outages_storms = outages_storms.sample(element_nb, random_state=random_seed)
    meaning_dict = {'begin_datetime': 'storm_start', 'end_datetime': 'storm_end', 'run_start_time_min': 'outage_start', 'run_start_time_max': 'outage_end'}
    sample_outages_storms = sample_outages_storms.rename(
        columns=meaning_dict
    )
    sample_outages_storms['one_day_before_storm'] = (sample_outages_storms['storm_start'] - pd.Timedelta(1, unit='D')).dt.date
    sample_outages_storms['one_day_after_storm'] = (sample_outages_storms['storm_end'] + pd.Timedelta(1, unit='D')).dt.date
    sample_outages_storms = sample_outages_storms.merge(counties[['fips_code_id', 'geometry']], on='fips_code_id')
    sample_outages_storms = gpd.GeoDataFrame(sample_outages_storms)
    sample_outages_storms['cntroid'] = sample_outages_storms.geometry.centroid
    sample_outages_storms['url__cntroid'] = sample_outages_storms.apply(get_url_params_from_row_point, desired_point='cntroid', axis=1)
    sample_outages_storms = sample_outages_storms[sample_outages_storms['url__cntroid'].notna()]
    t0 = time.perf_counter()
    cpu_counts = mp.cpu_count()
    print(cpu_counts)
    # Request meteorological data
    with mp.Pool(cpu_counts) as pool:
        sample_outages_storms['meteorological_info__cntroid'] = pool.map(
            meteorological_api.get_meteorological_info, sample_outages_storms.url__cntroid
        )
    
    t1 = time.perf_counter()
    print(f'time it took {t1-t0}')
    # Save data
    for row in sample_outages_storms.iterrows():
        save_meteorological_info_from_row_point(row)
    print('Done with saving.')
    # Wait.
    time.sleep(30)
    condition = condition - 1
    # Repeat.

Starting
20
time it took 463.5266085889998
Saving info at: ../data/raw/meteorological/163931_55075.json
Saving info at: ../data/raw/meteorological/153739_19153.json
Saving info at: ../data/raw/meteorological/101486_40031.json
Saving info at: ../data/raw/meteorological/154230_45029.json
Saving info at: ../data/raw/meteorological/140840_12009.json
Saving info at: ../data/raw/meteorological/114150_40015.json
Saving info at: ../data/raw/meteorological/160326_36117.json
Saving info at: ../data/raw/meteorological/153472_28111.json
Saving info at: ../data/raw/meteorological/180822_51059.json
Saving info at: ../data/raw/meteorological/138319_29047.json
Saving info at: ../data/raw/meteorological/108277_01089.json
Saving info at: ../data/raw/meteorological/155464_48439.json
Saving info at: ../data/raw/meteorological/164097_26111.json
Saving info at: ../data/raw/meteorological/185951_12086.json
Saving info at: ../data/raw/meteorological/171527_12086.json
Saving info at: ../data/raw/meteorological

In [313]:
# This is not needed...

In [309]:
sample_outages_storms = outages_storms.sample(element_nb, random_state=random_seed)
meaning_dict = {'begin_datetime': 'storm_start', 'end_datetime': 'storm_end', 'run_start_time_min': 'outage_start', 'run_start_time_max': 'outage_end'}
sample_outages_storms = sample_outages_storms.rename(
    columns=meaning_dict
)
sample_outages_storms['one_day_before_storm'] = (sample_outages_storms['storm_start'] - pd.Timedelta(1, unit='D')).dt.date
sample_outages_storms['one_day_after_storm'] = (sample_outages_storms['storm_end'] + pd.Timedelta(1, unit='D')).dt.date
sample_outages_storms = sample_outages_storms.merge(counties[['fips_code_id', 'geometry']], on='fips_code_id')
sample_outages_storms = gpd.GeoDataFrame(sample_outages_storms)

In [310]:
sample_outages_storms['rep_point'] = sample_outages_storms.geometry.representative_point()
#sample_outages_storms['up'] = sample_outages_storms.extract_unique_points()
sample_outages_storms['cntroid'] = sample_outages_storms.geometry.centroid
sample_outages_storms['boundary_sample_p'] = sample_outages_storms.geometry.boundary.sample_points(10, random_seed=random_seed)
sample_outages_storms['sample_p1'] = sample_outages_storms.geometry.sample_points(size=1, random_seed=random_seed)
sample_outages_storms['sample_p2'] = sample_outages_storms.geometry.sample_points(size=1, random_seed=random_seed+1)
sample_outages_storms['sample_p3'] = sample_outages_storms.geometry.sample_points(size=1, random_seed=random_seed+2)

In [311]:
# #!pip install pointpats
# s = sample_outages_storms.sample(1)
# ax = s.plot()
# s.rep_point.plot(ax=ax, color='r')
# #s.up.plot(ax=ax, color='g')
# s.cntroid.plot(ax=ax, color='y')
# s.boundary_sample_p.plot(ax=ax, color='orange')
# s.sample_p1.plot(ax=ax, color='gray')
# s.sample_p2.plot(ax=ax, color='gray')
# s.sample_p3.plot(ax=ax, color='gray')

In [291]:

    
#sample_outages_storms['url__rep_point'] = sample_outages_storms.apply(get_url_params_from_row_point, desired_point='rep_point', axis=1)
sample_outages_storms['url__cntroid'] = sample_outages_storms.apply(get_url_params_from_row_point, desired_point='cntroid', axis=1)
#sample_outages_storms['url__sp1'] = sample_outages_storms.apply(get_url_params_from_row_point, desired_point='sample_p1', axis=1)
#sample_outages_storms['url__sp2'] = sample_outages_storms.apply(get_url_params_from_row_point, desired_point='sample_p2', axis=1)
#sample_outages_storms['url__sp3'] = sample_outages_storms.apply(get_url_params_from_row_point, desired_point='sample_p3', axis=1)

In [292]:
sample_outages_storms = sample_outages_storms[sample_outages_storms['url__cntroid'].notna()]

In [293]:
#sample_outages_storms.url__rep_point.iloc[0]

In [294]:
sample_outages_storms

,EPISODE_ID,fips_code_id,episode_description,storm_start,storm_end,storm_duration,episode_fips_id,storm_caused_outage,outage_index_id,outage_start_minus_storm_start,...,one_day_before_storm,one_day_after_storm,geometry,rep_point,cntroid,boundary_sample_p,sample_p1,sample_p2,sample_p3,url__cntroid
1134,177512,41007,"Combination of snow, sleet and freezing rain.",2022-12-22 16:00:00,2022-12-24 13:30:00,45.500000,177512_41007,1.0,41007__0028,1.104167,...,2022-12-21,2022-12-25,"POLYGON ((-124.01136 46.23622, -124.002 46.237...",POINT (-123.64603 46.02537),POINT (-123.65572 45.99514),"MULTIPOINT (-123.99276 45.94733, -123.98143 46...",POINT (-123.61519 46.16772),POINT (-123.80363 45.82972),POINT (-123.45997 46.25325),https://power.larc.nasa.gov/api/temporal/hourl...
1297,143238,48491,Nearly all of South Central Texas had less tha...,2019-10-01 00:00:00,2019-10-31 23:59:00,743.983333,143238_48491,1.0,48491__0043,24.010417,...,2019-09-30,2019-11-01,"POLYGON ((-98.04989 30.62416, -97.998 30.72021...",POINT (-97.63453 30.66086),POINT (-97.60028 30.64775),"MULTIPOINT (-97.92403 30.60898, -97.86906 30.5...",POINT (-97.27337 30.62153),POINT (-97.32981 30.50738),POINT (-97.72788 30.59015),https://power.larc.nasa.gov/api/temporal/hourl...
1298,133333,39045,Several waves of low pressure moved along a st...,2019-02-06 11:48:00,2019-02-08 01:40:00,37.866667,133333_39045,1.0,39045__0026,1.497917,...,2019-02-05,2019-02-09,"POLYGON ((-82.83646 39.62786, -82.83549 39.637...",POINT (-82.61561 39.75553),POINT (-82.63066 39.75161),"MULTIPOINT (-82.83847 39.60729, -82.83312 39.6...",POINT (-82.54017 39.63577),POINT (-82.64963 39.88632),POINT (-82.61747 39.70291),https://power.larc.nasa.gov/api/temporal/hourl...
1323,122191,01123,A significant snowstorm impacted central Alaba...,2017-12-08 06:00:00,2017-12-09 05:00:00,23.000000,122191_01123,1.0,01123__0000,1.281250,...,2017-12-07,2017-12-10,"POLYGON ((-86.00917 33.09026, -85.97456 33.090...",POINT (-85.7985 32.79904),POINT (-85.79737 32.86253),"MULTIPOINT (-86.00765 32.84728, -86.00273 32.8...",POINT (-85.74843 32.91815),POINT (-85.62518 32.82237),POINT (-85.90016 32.8075),https://power.larc.nasa.gov/api/temporal/hourl...
1324,140999,55131,Persistent southwest winds and the passage of ...,2019-07-19 09:00:00,2019-07-20 14:20:00,29.333333,140999_55131,1.0,55131__0012,1.385417,...,2019-07-18,2019-07-21,"POLYGON ((-88.41871 43.32544, -88.41856 43.365...",POINT (-88.22326 43.36869),POINT (-88.2306 43.36852),"MULTIPOINT (-88.41858 43.23706, -88.40971 43.1...",POINT (-88.18444 43.53001),POINT (-88.22479 43.37107),POINT (-88.34135 43.41851),https://power.larc.nasa.gov/api/temporal/hourl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685,162889,04021,A shortwave trough rotating through the region...,2021-08-16 22:20:00,2021-08-17 09:00:00,10.666667,162889_04021,1.0,04021__0023,0.309028,...,2021-08-15,2021-08-18,"POLYGON ((-112.20359 32.63781, -112.20352 32.8...",POINT (-111.32618 32.91065),POINT (-111.34481 32.90444),"MULTIPOINT (-112.19055 33.26471, -112.03076 33...",POINT (-111.88107 32.64663),POINT (-111.93212 32.94542),POINT (-111.88854 32.9753),https://power.larc.nasa.gov/api/temporal/hourl...
1686,132631,39017,An upper level low pressure center tracked nor...,2018-11-14 23:00:00,2018-11-15 12:00:00,13.000000,132631_39017,1.0,39017__0043,0.406250,...,2018-11-13,2018-11-16,"POLYGON ((-84.81935 39.30945, -84.81745 39.391...",POINT (-84.57696 39.44808),POINT (-84.57555 39.43865),"MULTIPOINT (-84.81544 39.51715, -84.81214 39.5...",POINT (-84.45194 39.41688),POINT (-84.44585 39.40869),POINT (-84.36739 39.39598),https://power.larc.nasa.gov/api/temporal/hourl...
1687,99672,29113,A frontal boundary was slowly sinking south ac...,2015-06-25 17:25:00,2015-06-26 14:55:00,21.500000,99672_29113,1.0,29113__0000,0.305556,...,2015-06-24,2015-06-27,"POLYGON ((-91.26311 39.037, -91.26023 39.13984...",POINT (-90.98767 39.05039),POINT (-90.95986 39.05802),"MULTIPOINT (-91.18894 38

In [295]:
sample_outages_storms.url__cntroid.iloc[0]

'https://power.larc.nasa.gov/api/temporal/hourly/point?parameters=T2M,ALLSKY_SFC_SW_DWN,PS,WS50M,ALLSKY_SFC_LW_DWN,WD50M,PRECTOTCORR,GWETPROF,CLRSKY_SFC_SW_DWN,QV10M,RHOA,T10M,TO3,TQV,Z0M,TOA_SW_DWN,RH2M,WS2M,CLRSKY_SFC_LW_DWN,DISPH&community=RE&longitude=-123.6557206172946&latitude=45.995139447263576&start=20221221&end=20221225&format=JSON'

In [296]:
#sample_outages_storms.url__sp1.iloc[0]

In [297]:
#%%time
#sample_outages_storms['meteorological_info__rep_point'] = sample_outages_storms.url__rep_point.apply(meteorological_api.get_meteorological_info)

In [298]:
#%%time
#sample_outages_storms['meteorological_info__url__cntroid'] = sample_outages_storms.url__cntroid.apply(meteorological_api.get_meteorological_info)

In [299]:
%%time
sample_outages_storms['meteorological_info__cntroid'] = sample_outages_storms.url__cntroid.swifter.allow_dask_on_strings(enable=True).apply(meteorological_api.get_meteorological_info)

Dask Apply:   0%|          | 0/41 [00:00<?, ?it/s]

/mnt/c/Users/52333/Documents/projects/dynamic-rythms/venv/lib/python3.10/site-packages/dask/utils.py:1226: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)
/mnt/c/Users/52333/Documents/projects/dynamic-rythms/venv/lib/python3.10/site-packages/dask/utils.py:1226: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)
/mnt/c/Users/52333/Documents/projects/dynamic-rythms/venv/lib/python3.10/site-packages/dask/utils.py:1226: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(

CPU times: user 770 ms, sys: 182 ms, total: 953 ms
Wall time: 14min 7s


In [300]:
#sample_outages_storms['meteorological_info__cntroid']

In [301]:
#rep_point = pd.DataFrame(sample_outages_storms.meteorological_info__rep_point.iloc[9]['properties']['parameter'])
centroid = pd.DataFrame(sample_outages_storms.meteorological_info__cntroid.iloc[9]['properties']['parameter'])
#rep_point-centroid


In [302]:
#sample_outages_storms['meteorological_info__cntroid'].iloc[0].get('properties')

In [303]:
def save_meteorological_info_from_row_point(storm_outage_row_iterated):
    idx, storm_outage_row = storm_outage_row_iterated
    identifier = storm_outage_row['episode_fips_id']
    meteorological_info = storm_outage_row['meteorological_info__cntroid']
    validate = meteorological_info.get('properties')
    save_path = f'../data/raw/meteorological/{identifier}.json'
    filexists = utils.check_if_filepath_exists(save_path)
    if (not filexists) and (validate is not None):
        meteorological_api.save_meteorological_information(information=meteorological_info, path=save_path)
        print(f'Saving info at: {save_path}')
        return 1
    return np.nan

In [304]:
for row in sample_outages_storms.iterrows():
    save_meteorological_info_from_row_point(row)

Saving info at: ../data/raw/meteorological/177512_41007.json
Saving info at: ../data/raw/meteorological/143238_48491.json
Saving info at: ../data/raw/meteorological/133333_39045.json
Saving info at: ../data/raw/meteorological/122191_01123.json
Saving info at: ../data/raw/meteorological/157072_48273.json
Saving info at: ../data/raw/meteorological/164104_06067.json
Saving info at: ../data/raw/meteorological/155446_48005.json
Saving info at: ../data/raw/meteorological/112444_12109.json
Saving info at: ../data/raw/meteorological/97063_24033.json
Saving info at: ../data/raw/meteorological/133253_17197.json
Saving info at: ../data/raw/meteorological/93589_21111.json
Saving info at: ../data/raw/meteorological/148533_39165.json
Saving info at: ../data/raw/meteorological/126901_12021.json
Saving info at: ../data/raw/meteorological/184949_49049.json
Saving info at: ../data/raw/meteorological/164151_06087.json
Saving info at: ../data/raw/meteorological/99116_48005.json
Saving info at: ../data/raw

In [305]:
sample_outages_storms

,EPISODE_ID,fips_code_id,episode_description,storm_start,storm_end,storm_duration,episode_fips_id,storm_caused_outage,outage_index_id,outage_start_minus_storm_start,...,one_day_after_storm,geometry,rep_point,cntroid,boundary_sample_p,sample_p1,sample_p2,sample_p3,url__cntroid,meteorological_info__cntroid
1134,177512,41007,"Combination of snow, sleet and freezing rain.",2022-12-22 16:00:00,2022-12-24 13:30:00,45.500000,177512_41007,1.0,41007__0028,1.104167,...,2022-12-25,"POLYGON ((-124.01136 46.23622, -124.002 46.237...",POINT (-123.64603 46.02537),POINT (-123.65572 45.99514),"MULTIPOINT (-123.99276 45.94733, -123.98143 46...",POINT (-123.61519 46.16772),POINT (-123.80363 45.82972),POINT (-123.45997 46.25325),https://power.larc.nasa.gov/api/temporal/hourl...,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1297,143238,48491,Nearly all of South Central Texas had less tha...,2019-10-01 00:00:00,2019-10-31 23:59:00,743.983333,143238_48491,1.0,48491__0043,24.010417,...,2019-11-01,"POLYGON ((-98.04989 30.62416, -97.998 30.72021...",POINT (-97.63453 30.66086),POINT (-97.60028 30.64775),"MULTIPOINT (-97.92403 30.60898, -97.86906 30.5...",POINT (-97.27337 30.62153),POINT (-97.32981 30.50738),POINT (-97.72788 30.59015),https://power.larc.nasa.gov/api/temporal/hourl...,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1298,133333,39045,Several waves of low pressure moved along a st...,2019-02-06 11:48:00,2019-02-08 01:40:00,37.866667,133333_39045,1.0,39045__0026,1.497917,...,2019-02-09,"POLYGON ((-82.83646 39.62786, -82.83549 39.637...",POINT (-82.61561 39.75553),POINT (-82.63066 39.75161),"MULTIPOINT (-82.83847 39.60729, -82.83312 39.6...",POINT (-82.54017 39.63577),POINT (-82.64963 39.88632),POINT (-82.61747 39.70291),https://power.larc.nasa.gov/api/temporal/hourl...,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1323,122191,01123,A significant snowstorm impacted central Alaba...,2017-12-08 06:00:00,2017-12-09 05:00:00,23.000000,122191_01123,1.0,01123__0000,1.281250,...,2017-12-10,"POLYGON ((-86.00917 33.09026, -85.97456 33.090...",POINT (-85.7985 32.79904),POINT (-85.79737 32.86253),"MULTIPOINT (-86.00765 32.84728, -86.00273 32.8...",POINT (-85.74843 32.91815),POINT (-85.62518 32.82237),POINT (-85.90016 32.8075),https://power.larc.nasa.gov/api/temporal/hourl...,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1324,140999,55131,Persistent southwest winds and the passage of ...,2019-07-19 09:00:00,2019-07-20 14:20:00,29.333333,140999_55131,1.0,55131__0012,1.385417,...,2019-07-21,"POLYGON ((-88.41871 43.32544, -88.41856 43.365...",POINT (-88.22326 43.36869),POINT (-88.2306 43.36852),"MULTIPOINT (-88.41858 43.23706, -88.40971 43.1...",POINT (-88.18444 43.53001),POINT (-88.22479 43.37107),POINT (-88.34135 43.41851),https://power.larc.nasa.gov/api/temporal/hourl...,{'message': 'Internal server error'}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685,162889,04021,A shortwave trough rotating through the region...,2021-08-16 22:20:00,2021-08-17 09:00:00,10.666667,162889_04021,1.0,04021__0023,0.309028,...,2021-08-18,"POLYGON ((-112.20359 32.63781, -112.20352 32.8...",POINT (-111.32618 32.91065),POINT (-111.34481 32.90444),"MULTIPOINT (-112.19055 33.26471, -112.03076 33...",POINT (-111.88107 32.64663),POINT (-111.93212 32.94542),POINT (-111.88854 32.9753),https://power.larc.nasa.gov/api/temporal/hourl...,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1686,132631,39017,An upper level low pressure center tracked nor...,2018-11-14 23:00:00,2018-11-15 12:00:00,13.000000,132631_39017,1.0,39017__0043,0.406250,...,2018-11-16,"POLYGON ((-84.81935 39.30945, -84.81745 39.391...",POINT (-84.57696 39.44808),POINT (-84.57555 39.43865),"MULTIPOINT (-84.81544 39.51715, -84.81214 39.5...",POINT (-84.45194 39.41688),POINT (-84.44585 39.40869),POINT (-84.36739 39.39598),https://power.larc.nasa.gov/api/temporal/hourl...,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1687,99672,29113,A frontal boun